In [21]:
import pandas as pd

# Membaca dataset
file_path = '../data/Data_MNL_siang.xlsx'
data_trip = pd.read_excel(file_path)

# Menampilkan informasi dasar dari dataset
data_trip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11540 entries, 0 to 11539
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   driver_id                        11540 non-null  object
 1   loc_jumlah_taman                 11540 non-null  int64 
 2   loc_jumlah_pickup                11540 non-null  int64 
 3   loc_jumlah_sekolah               11540 non-null  int64 
 4   loc_jumlah_ofm                   11540 non-null  int64 
 5   jumlah_faskes                    11540 non-null  int64 
 6   loc_jumlah_hotel                 11540 non-null  int64 
 7   loc_jml_b_permukiman             11540 non-null  int64 
 8   loc_populasi                     11540 non-null  int64 
 9   loc_jumlah_terminal              11540 non-null  int64 
 10  trip_jenis_pesanan_Ride          11540 non-null  bool  
 11  trip_jenis_pesanan_Kurir Barang  11540 non-null  bool  
 12  loc_jumlah_pasar                

In [22]:
data_trip.head()

,driver_id,loc_jumlah_taman,loc_jumlah_pickup,loc_jumlah_sekolah,loc_jumlah_ofm,jumlah_faskes,loc_jumlah_hotel,loc_jml_b_permukiman,loc_populasi,loc_jumlah_terminal,trip_jenis_pesanan_Ride,trip_jenis_pesanan_Kurir Barang,loc_jumlah_pasar,sosdem_usia,location_code
0,ikialink8@gmail.com,0,63,5,68,0,0,1582,10930,0,True,False,0,34,59
1,ikialink8@gmail.com,3,1972,32,163,6,21,829,2255,0,True,False,1,34,40
2,ikialink8@gmail.com,0,237,10,34,1,5,2387,26037,0,True,False,0,34,81
3,ikialink8@gmail.com,0,109,5,36,1,6,3421,19630,0,True,False,1,34,54
4,ikialink8@gmail.com,0,50,2,18,1,0,3045,19960,0,True,False,0,34,6


In [23]:
data_trip.columns

Index(['driver_id', 'loc_jumlah_taman', 'loc_jumlah_pickup',
       'loc_jumlah_sekolah', 'loc_jumlah_ofm', 'jumlah_faskes',
       'loc_jumlah_hotel', 'loc_jml_b_permukiman', 'loc_populasi',
       'loc_jumlah_terminal', 'trip_jenis_pesanan_Ride',
       'trip_jenis_pesanan_Kurir Barang', 'loc_jumlah_pasar', 'sosdem_usia',
       'location_code'],
      dtype='object')

In [24]:
data_trip.rename(columns={'trip_jenis_pesanan_Kurir Barang': 'trip_jenis_pesanan_Kurir_Barang'}, inplace=True)

In [25]:
from sklearn.preprocessing import StandardScaler

# Asumsi bahwa data_trip adalah DataFrame Anda
features = ['loc_jumlah_taman', 'loc_jumlah_pickup', 'loc_jumlah_sekolah', 
            'loc_jumlah_ofm', 'jumlah_faskes', 'loc_jumlah_hotel', 'loc_jml_b_permukiman', 
            'loc_populasi', 'loc_jumlah_terminal', 'trip_jenis_pesanan_Ride', 
            'trip_jenis_pesanan_Kurir_Barang', 'loc_jumlah_pasar', 'sosdem_usia']

# Mengisolasi features
X = data_trip[features]

# Membuat StandardScaler dan melakukan transformasi
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Menggabungkan kembali data yang telah diskalakan ke DataFrame
data_trip_scaled = data_trip.copy()
data_trip_scaled[features] = X_scaled

In [26]:
# Definisi Fungsi Utilitas
def utility_function(driver_id, location_id, beta, data_trip):
    # Ambil baris data untuk pengemudi dan lokasi tertentu
    data_row = data_trip[(data_trip.driver_id == driver_id) & (data_trip.location_code == location_id)]

    # Jika kombinasi tidak ditemukan, kembalikan utilitas 0 atau nilai default
    if data_row.empty:
        return 0

    data_row = data_row.iloc[0]

    # Hitung utilitas berdasarkan variabel
    utility = (beta[0] * data_row.loc_jumlah_taman + 
               beta[1] * data_row.loc_jumlah_pickup + 
               beta[2] * data_row.loc_jumlah_sekolah + 
               beta[3] * data_row.loc_jumlah_ofm + 
               beta[4] * data_row.jumlah_faskes + 
               beta[5] * data_row.loc_jumlah_hotel + 
               beta[6] * data_row.loc_jml_b_permukiman + 
               beta[7] * data_row.loc_populasi + 
               beta[8] * data_row.loc_jumlah_terminal + 
               beta[9] * data_row.trip_jenis_pesanan_Ride + 
               beta[10] * data_row.trip_jenis_pesanan_Kurir_Barang + 
               beta[11] * data_row.loc_jumlah_pasar + 
               beta[12] * data_row.sosdem_usia)  # N adalah jumlah total variabel
    return utility

In [38]:
def log_likelihood(beta, data_trip):
    total_log_likelihood = 0

    for driver_id in set(data_trip.driver_id):
        driver_data = data_trip[data_trip.driver_id == driver_id]
        for index, trip in driver_data.iterrows():
            # print(len(set(driver_data.location_code)))
            # Hitung utilitas untuk pilihan ini
            util = utility_function(driver_id, trip['location_code'], beta, data_trip)

            # Hitung total utilitas untuk semua lokasi yang mungkin dipilih oleh pengemudi ini
            sum_exp_utilities = sum(np.exp(utility_function(driver_id, loc, beta, data_trip)) for loc in set(driver_data.location_code))

            # Tambahkan logaritma dari probabilitas pilihan ini ke total log-likelihood
            total_log_likelihood += util - np.log(sum_exp_utilities)

    return -total_log_likelihood

In [6]:
def log_likelihood(beta, data_trip):
    total_log_likelihood = 0

    # Iterasi melalui setiap pengemudi
    for driver_id in set(data_trip.driver_id):
        # Iterasi melalui setiap lokasi
        for location_id in set(data_trip.location_code):
            print(driver_id, location_id)
            # Menghitung utilitas untuk pengemudi dan lokasi tertentu
            util = utility_function(driver_id, location_id, beta, data_trip)

            # Menghitung ekponensial utilitas untuk pengemudi dan lokasi tertentu
            exp_util = np.exp(util)

            # Menghitung total ekponensial utilitas untuk semua lokasi bagi pengemudi tertentu
            sum_exp_util = sum(np.exp(utility_function(driver_id, loc, beta, data_trip)) for loc in set(data_trip.location_code))

            # Probabilitas pilihan lokasi ini oleh pengemudi
            prob = exp_util / sum_exp_util

            # Menambahkan log probabilitas ke log likelihood total jika pengemudi memilih lokasi ini
            if not data_trip[(data_trip.driver_id == driver_id) & (data_trip.location_code == location_id)].empty:
                total_log_likelihood += np.log(prob)

    return -total_log_likelihood  # Minimisasi log-likelihood negatif


In [39]:
from scipy.optimize import minimize
import numpy as np

# Estimasi Model dengan Toleransi Konvergensi
initial_beta = np.zeros(13)  # Sesuaikan jumlah ini sesuai jumlah variabel
result = minimize(log_likelihood, initial_beta, args=(data_trip_scaled), method='BFGS')
 
# Menampilkan Hasil
print("Optimized Beta Parameters:", result.x)

KeyboardInterrupt: 

In [12]:
# Menghitung L(0)
L0 = log_likelihood(np.zeros_like(initial_beta), data_trip)

# Menghitung LL (log-likelihood dari model)
LL = result.fun

# Jumlah parameter model
k = len(result.x)

# Menghitung Rho-Square
rho_square = 1 - (LL / L0)

# Menghitung Rho-Square yang Disesuaikan
rho_square_adj = 1 - ((LL - k) / L0)

# Menghitung T-Statistic
# Periksa dulu apakah matriks Hessian tersedia
if hasattr(result, 'hess_inv'):
    std_err = np.sqrt(np.diag(np.linalg.inv(result.hess_inv)))
    t_stats = result.x / std_err
else:
    std_err = None
    t_stats = None

# Menampilkan Hasil
print("Optimized Beta Parameters:", result.x)
print("Log-Likelihood:", LL)
print("L(0):", L0)
print("Rho-Square:", rho_square)
print("Adjusted Rho-Square:", rho_square_adj)
print("T-Statistic:", t_stats)

Optimized Beta Parameters: [ -417299.48131926  -653389.80966023   695694.66083869 -1001201.71815899
  -310398.95876973 -1692486.04937483 -2658024.80233984   741367.31929913
   -88493.23420259  -101531.06926621   -55365.30702316  -199024.46439121
  4457422.59036461]
Log-Likelihood: -54253679850064.32
L(0): 47432.583368181455
Rho-Square: 1143806135.886985
Adjusted Rho-Square: 1143806135.887259
T-Statistic: [-1.76004303e+05 -8.15907037e+02  5.24561641e+04 -1.71578347e+04
 -1.21084530e+05 -1.50001405e+05 -7.11903817e+02  5.22639201e+01
 -6.11207894e+04 -1.00416238e+05 -5.43618840e+04 -1.44524741e+05
  3.74175309e+06]
